## Importamos librerias

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
PATH = 'data/'

## Cargamos los datos

    En este caso vamos a realizar un análisis de la jornada 4 para poder detectar que fichajes serían los más optimos de cara a la jornada del campeonato para tratar de obtener la mejor puntuación posible.

In [5]:
journey = pd.read_csv(PATH+ 'comunio_J4.csv')
clas = pd.read_excel(PATH + 'classification_J_4.xlsx')

In [6]:
journey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Team_id              518 non-null    int64  
 1   Team                 518 non-null    object 
 2   Player               518 non-null    object 
 3   Position             518 non-null    object 
 4   Matchs               518 non-null    int64  
 5   On_start_%           518 non-null    int64  
 6   Goals                518 non-null    int64  
 7   Assists              518 non-null    int64  
 8   Total_Points         518 non-null    int64  
 9   Points_Average       518 non-null    float64
 10  Last_5_games_points  518 non-null    object 
 11  J_4                  518 non-null    int64  
 12  J_3                  518 non-null    int64  
 13  J_2                  518 non-null    int64  
 14  J_1                  518 non-null    int64  
 15  J_0                  518 non-null    int

In [7]:
clas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20 non-null     int64 
 1   Position    20 non-null     int64 
 2   Team        20 non-null     object
 3   Games       20 non-null     int64 
 4   Won         20 non-null     int64 
 5   Draw        20 non-null     int64 
 6   Lost        20 non-null     int64 
 7   Goals S.    20 non-null     int64 
 8   Goals R.    20 non-null     int64 
 9   GD          20 non-null     int64 
 10  PTS         20 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.8+ KB


*Con los datos cargados vamos a realizar las transformaciones y filtros necesarios para obtener la tabla que necesitamos para el dashboard*

In [10]:
journey = journey.drop('Last_5_games_points', axis=1)

In [12]:
clas = clas.drop('Unnamed: 0', axis=1)

Filtramos el df para extraer los mejores jugadores por posición

In [13]:
gk = journey.loc[journey['Position']=='PT'].sort_values('Total_Points', ascending=False)[:5]
df = journey.loc[journey['Position']=='DF'].sort_values('Total_Points', ascending=False)[:10]
md = journey.loc[journey['Position']=='MD'].sort_values('Total_Points', ascending=False)[:10]
dl = journey.loc[journey['Position']=='DL'].sort_values('Total_Points', ascending=False)[:10]

In [14]:
pos = [gk, df, md, dl]

In [15]:
top_players = pd.DataFrame()

for p in pos:
    top_players = pd.concat([top_players, pd.DataFrame(p)], axis=0)

top_players.head()

,Team_id,Team,Player,Position,Matchs,On_start_%,Goals,Assists,Total_Points,Points_Average,J_4,J_3,J_2,J_1,J_0,Avg_last_5_games,Value
281,12,Villarreal CF,Rulli,PT,4,100,0,0,36,9.0,12,11,8,5,0,7.2,4990000
101,5,FC Barcelona,Ter Stegen,PT,4,100,0,0,32,8.0,8,7,7,10,0,6.4,5880000
387,16,UD Almería,Fernando Martínez,PT,4,100,0,0,29,7.3,12,4,7,6,0,5.8,3310000
226,10,Real Betis Balompié,Rui Silva,PT,4,100,0,0,26,6.5,7,9,8,2,0,5.2,3280000
415,17,Elche CF,Édgar Badía,PT,4,100,0,0,23,5.8,2,7,10,4,0,4.6,2900000


## Ahora vamos ha realizar un poco ingeniería de datos para obtener algunas columnas que nos puedas ayudar al estudio

*En primer lugar vamos a añadir una columna para obtener el precio por puntos totales de cada jugador*

In [16]:
top_players['Price_per_Total_Points'] = (top_players['Value']/top_players.Total_Points).apply(lambda x: '{0:.2f}'.format(x))

*Ahora hacemos lo mismo pero únicamente calculando el valor por punto obtenido en las últimas cinco jornadas*

In [17]:
top_players['Price_per_Points_Last_4_J'] = (top_players['Value']/ (top_players['J_4']
                                                    + top_players['J_3']
                                                    + top_players['J_2']
                                                    + top_players['J_1'])).apply(lambda x: '{0:.2f}'.format(x))
    

In [18]:
top_players.Avg_last_5_games = round((top_players.J_4+top_players.J_3+top_players.J_2+top_players.J_1)/4, 1)

In [19]:
top_players.head()

,Team_id,Team,Player,Position,Matchs,On_start_%,Goals,Assists,Total_Points,Points_Average,J_4,J_3,J_2,J_1,J_0,Avg_last_5_games,Value,Price_per_Total_Points,Price_per_Points_Last_4_J
281,12,Villarreal CF,Rulli,PT,4,100,0,0,36,9.0,12,11,8,5,0,9.0,4990000,138611.11,138611.11
101,5,FC Barcelona,Ter Stegen,PT,4,100,0,0,32,8.0,8,7,7,10,0,8.0,5880000,183750.00,183750.00
387,16,UD Almería,Fernando Martínez,PT,4,100,0,0,29,7.3,12,4,7,6,0,7.2,3310000,114137.93,114137.93
226,10,Real Betis Balompié,Rui Silva,PT,4,100,0,0,26,6.5,7,9,8,2,0,6.5,3280000,126153.85,126153.85
415,17,Elche CF,Édgar Badía,PT,4,100,0,0,23,5.8,2,7,10,4,0,5.8,2900000,126086.96,126086.96


### Una vez que tenemos la tabla con los datos la exportamos a excel para poder cargarla en Tableau 

In [20]:
with pd.ExcelWriter(PATH + 'comunio_tableau_J4.xlsx') as writer:  
    top_players.to_excel(writer, sheet_name='top_players', index=False)
    clas.to_excel(writer, sheet_name='classification', index=False)